In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 1, 4), datetime.date(2022, 1, 3))

In [2]:
today = today - timedelta(days=5)
wkago = today - timedelta(days=6)
today, wkago

(datetime.date(2021, 12, 30), datetime.date(2021, 12, 24))

### Weekly process or when stataus changes.

In [3]:
names = 'MCS PTTGC'.split()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'MCS', 'PTTGC'"

### Restart and Run All Cells

In [4]:
sqlDel = """
DELETE FROM sales
WHERE name IN (%s)"""
sqlDel = sqlDel % in_p
rp = conlite.execute(sqlDel)
rp.rowcount

0

In [5]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)

df = pd.read_sql(sql, const)
df.shape


SELECT * 
FROM price 
WHERE date >= '2021-01-03' AND name IN ('MCS', 'PTTGC') 
ORDER BY name, date


(482, 7)

In [6]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [7]:
sql = """
SELECT name,status 
FROM stocks 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,status
name,
MCS,T
PTTGC,U


### Create monitors from stocks

In [8]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,status
name,
BGC,O
BGRIM,I
DCC,I
DOHOME,B
GLOBAL,B
IMH,B
IVL,I
KBANK,I
KCE,B


In [9]:
monitors.shape[0]

23

In [10]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [11]:
sql = """
SELECT status, COUNT(*) AS items 
FROM stocks 
WHERE name IN (%s)
GROUP BY status
ORDER BY status
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,status,items
0,T,1
1,U,1


In [12]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [13]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(116, 1)

In [14]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(116, 3)

### Names in this step must be manually added to price-uploads.csv

In [15]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,status,_merge


In [16]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(93, 3)

In [17]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
DOHOME,B,SET100
GLOBAL,B,SET50
IMH,B,mai
KCE,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
SIS,B,SET
STA,B,SET50


In [18]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [19]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BGC,O
DOHOME,B
GLOBAL,B
IMH,B
KCE,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O


In [20]:
buy_candidates.shape[0]

16

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BGRIM,I
DCC,I
IVL,I
KBANK,I
RJH,I
SAT,I
TOP,I


In [22]:
sell_candidates.shape[0]

7